In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [149]:
num_epochs = 100
num_classes = 3
batch_size = 4
learning_rate = 0.001

In [150]:
data_dir = './Spects/train'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.RandomResizedCrop(150),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.RandomResizedCrop(150),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

['Adult', 'Child', 'Irrelevant']


In [151]:
## Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

device(type='cuda')

In [161]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 37* 37, 1024)
        self.fc2 = nn.Linear(1024, 3)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, 32 * 37* 37)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Net().to(device)


In [162]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [163]:
## Train the Network

for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i, (images, labels) in enumerate(trainloader):
        # get the inputs
        images = images.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 2.173
[2,   200] loss: 2.094
[3,   200] loss: 2.116
[4,   200] loss: 2.114
[5,   200] loss: 2.099
[6,   200] loss: 2.113
[7,   200] loss: 2.101
[8,   200] loss: 2.096
[9,   200] loss: 2.119
[10,   200] loss: 2.104
[11,   200] loss: 2.104
[12,   200] loss: 2.105
[13,   200] loss: 2.098
[14,   200] loss: 2.111
[15,   200] loss: 2.100
[16,   200] loss: 2.111
[17,   200] loss: 2.100
[18,   200] loss: 2.111
[19,   200] loss: 2.108
[20,   200] loss: 2.104
[21,   200] loss: 2.091
[22,   200] loss: 2.112
[23,   200] loss: 2.107
[24,   200] loss: 2.101
[25,   200] loss: 2.094
[26,   200] loss: 2.121
[27,   200] loss: 2.095
[28,   200] loss: 2.108
[29,   200] loss: 2.107
[30,   200] loss: 2.111
[31,   200] loss: 2.103
[32,   200] loss: 2.105
[33,   200] loss: 2.090
[34,   200] loss: 2.102
[35,   200] loss: 2.114
[36,   200] loss: 2.098
[37,   200] loss: 2.109
[38,   200] loss: 2.116
[39,   200] loss: 2.103
[40,   200] loss: 2.107
[41,   200] loss: 2.102
[42,   200] loss: 2.108
[

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))